# Poem-to-Dataset Generator (OpenRouter API)

This notebook reads poem verses from `poem_condense.csv` and generates a semantically grounded synthetic dataset using OpenRouter API.

**Pipeline:**
1. Load poem verses from CSV
2. For each verse, prompt an LLM to generate:
   - Modern interpretation (`meaning`)
   - Neutral queries (5 diverse prompts)
   - User queries (5 diverse persona-based prompts)
3. Save results to `poem_finetune.jsonl`

## Cell 1: Imports

In [1]:
import os
import json
import time
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
from typing import Dict, Optional, Tuple
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
import json_repair

## Cell 2: Configuration

**Important:** Set your OpenRouter API key in the environment variable `OPEN_ROUTER_API_KEY` or directly in the config below.

In [2]:
# Configuration
CONFIG = {
    "api_key": os.getenv("OPEN_ROUTER_API_KEY", "YOUR_API_KEY_HERE"),
    "base_url": "https://openrouter.ai/api/v1",
    "model": "mistralai/mistral-small-creative",
    "temperature": 0.7,
    "max_tokens": 10_000,
    "http_referer": "https://github.com/acitea/fine-poems",
    "x_title": "Poem Fine-Tuning Data Generator",
    "input_file": "../data/poem_condense.csv",
    "output_file": "../data/poem_finetune.jsonl",
    "failed_output_file": "../data/poem_finetune_failed.jsonl",
    "max_retries": 3,
    "retry_delay": 2,
    "concurrency": 20,
}

# Validate API Key
if CONFIG["api_key"] == "YOUR_API_KEY_HERE":
    print("⚠️  WARNING: Please set your OPEN_ROUTER_API_KEY!")
else:
    print("✅ API Key loaded successfully")

✅ API Key loaded successfully


## Cell 3: Initialize OpenAI Client

In [3]:
# Initialize OpenRouter client
client = OpenAI(
    base_url=CONFIG["base_url"],
    api_key=CONFIG["api_key"],
)

print(f"🔌 Connected to OpenRouter (Model: {CONFIG['model']})")

🔌 Connected to OpenRouter (Model: mistralai/mistral-small-creative)


## Cell 4: API Utility Functions

In [4]:
def generate_dataset_entry(poem_verse: str, retries: int = CONFIG["max_retries"]) -> Tuple[Optional[Dict], Optional[Dict]]:
    """
    Generate a dataset entry for a given poem verse using the LLM.

    Returns:
        (success_record, failure_record)
        - success_record: {"poem_verse": ..., "data": ...} when successful
        - failure_record: diagnostic payload when all retries fail
    """
    prompt = f"""I will provide a poem verse from Project Gutenberg. Return a JSON object with:

* `meaning`: A modern, clear, style-neutral version of the sentence.
* `queries`: A dictionary with two keys:
  * `neutral`: A list of strings of 5 diverse prompts in plain English that would naturally result in the target meaning.
  * `user`: A list of strings of 5 diverse prompts where the user adopts a specific persona or context that would trigger the assistant to answer using the provided poem verse.

**Constraint:** Ensure prompts vary in length from a single short sentence to a detailed paragraph.

**Poem Verse:**
"{poem_verse}"

Return ONLY the JSON object, no additional text."""

    system_prompt = """You are an expert literary analyst and synthetic dataset architect specializing in LLM fine-tuning data generation. 

Your task is to transform classical poetry into high-quality training examples for language models. You must:

1. **Semantic Grounding**: Extract the core meaning from archaic or poetic language and express it in clear, contemporary terms.

2. **Query Diversity**: Generate prompts that vary significantly in:
   - Length (from 5 words to 100+ words)
   - Complexity (simple questions to nuanced scenarios)
   - Formality (casual to academic)
   - Specificity (general inquiries to precise requests)

3. **Persona Variation**: For user queries, randomly invent diverse personas with varying backgrounds, intentions, and contexts. Be creative and unpredictable—avoid repeating similar persona types. Each persona should feel unique and authentic. Think broadly: different ages, professions, cultural contexts, emotional states, levels of expertise, communication styles, and reasons for asking.
   
4. **Naturalness**: Ensure all queries sound like authentic human requests that would organically lead to the target response. Vary sentence structure, vocabulary, and tone across all queries.

5. **Format Compliance**: Return ONLY a valid JSON object with the exact schema requested. No markdown formatting, no explanatory text."""
    last_error = None
    last_raw_response = None

    for attempt in range(retries):
        try:
            response = client.chat.completions.create(
                model=CONFIG["model"],
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": prompt},
                ],
                temperature=CONFIG["temperature"],
                max_tokens=CONFIG["max_tokens"],
            )

            raw_response = (response.choices[0].message.content or "").strip()
            last_raw_response = raw_response

            parsed = parse_json_response(raw_response)
            if parsed and validate_response_structure(parsed):
                return {"poem_verse": poem_verse, "data": parsed}, None

            last_error = "Invalid response structure"

        except Exception as e:
            last_error = f"{type(e).__name__}: {str(e)}"

        if attempt < retries - 1:
            time.sleep(CONFIG["retry_delay"] * (attempt + 1))

    failure_record = {
        "poem_verse": poem_verse,
        "last_error": last_error,
        "last_raw_response": last_raw_response,
        "timestamp": time.time(),
    }
    return None, failure_record


def parse_json_response(raw_response: str) -> Optional[Dict]:
    """
    Parse JSON from LLM response using json_repair for robust extraction.
    Handles markdown, control chars, incomplete JSON, and common formatting issues.
    """
    # Remove markdown code blocks
    raw_response = re.sub(r"^```json\s*", "", raw_response, flags=re.MULTILINE)
    raw_response = re.sub(r"^```\s*", "", raw_response, flags=re.MULTILINE)
    raw_response = re.sub(r"```\s*$", "", raw_response, flags=re.MULTILINE)
    raw_response = raw_response.strip()

    # Extract JSON object boundaries
    start_idx = raw_response.find("{")
    end_idx = raw_response.rfind("}")

    if start_idx == -1 or end_idx == -1 or end_idx <= start_idx:
        print(f"⚠️  No JSON object found in response")
        print(f"Full response:\n{raw_response}")
        return None

    json_str = raw_response[start_idx : end_idx + 1]

    try:
        repaired = json_repair.loads(json_str)
        return repaired
    except Exception as e:
        print(f"⚠️  JSON repair and parse failed: {str(e)}")
        print(f"Full extracted JSON:\n{json_str}")
        return None

def validate_response_structure(data: Dict) -> bool:
    """
    Validate that the response has the required structure.
    """
    required_keys = ["meaning", "queries"]
    if not all(key in data for key in required_keys):
        return False

    queries = data.get("queries", {})
    if not isinstance(queries, dict):
        return False

    if "neutral" not in queries or "user" not in queries:
        return False

    if not isinstance(queries["neutral"], list) or len(queries["neutral"]) != 5:
        return False

    if not isinstance(queries["user"], list) or len(queries["user"]) != 5:
        return False

    return True


print("✅ Utility functions loaded")

✅ Utility functions loaded


## Cell 5: Load Input Data

In [5]:
# Load poem verses from CSV
df = pd.read_csv(CONFIG["input_file"])

print(f"📊 Loaded {len(df)} poem verses from {CONFIG['input_file']}")
print(f"Columns: {list(df.columns)}")
print(f"\nFirst few rows:")
print(df.head())

# Identify the column containing poem text
# Adjust this based on your CSV structure
if 'verse' in df.columns:
    poem_column = 'verse'
elif 'text' in df.columns:
    poem_column = 'text'
elif 'poem' in df.columns:
    poem_column = 'poem'
else:
    poem_column = df.columns[1]  # Use first column as fallback
    
print(f"\n🎯 Using column '{poem_column}' for poem verses")

📊 Loaded 199002 poem verses from ../data/poem_condense.csv
Columns: ['Unnamed: 0', 'Verse', 'Meter', 'char_count']

First few rows:
   Unnamed: 0                                              Verse   Meter  \
0           0          ah why this boding start this sudden pain  iambic   
1           1   that wings my pulse and shoots from vein to vein  iambic   
2           2          what mean regardless of yon midnight bell  iambic   
3           3     these earthborn visions saddening o'er my cell  iambic   
4           4  what strange disorder prompts these thoughts t...  iambic   

   char_count  
0           6  
1           6  
2           6  
3           6  
4           6  

🎯 Using column 'Verse' for poem verses


## Cell 6: Main Processing Loop

This cell processes each poem verse and saves results incrementally to prevent data loss.

In [ ]:
# Initialize output files (clear if exist)
with open(CONFIG["output_file"], "w") as f:
    pass
with open(CONFIG["failed_output_file"], "w") as f:
    pass

# Tracking
successful = 0
failed = 0
failed_verses = []

# Build task list first (skip empty verses)
tasks = []
for idx, row in df.iterrows():
    poem_verse = row[poem_column]
    if pd.isna(poem_verse) or not str(poem_verse).strip():
        continue
    tasks.append((int(idx), str(poem_verse)))

print(f"🚀 Running {len(tasks)} requests with concurrency={CONFIG['concurrency']}")

# Execute concurrently and write results as they complete
with ThreadPoolExecutor(max_workers=CONFIG["concurrency"]) as executor:
    future_to_task = {
        executor.submit(generate_dataset_entry, verse): (idx, verse)
        for idx, verse in tasks
    }

    for future in tqdm(as_completed(future_to_task), total=len(future_to_task), desc="Processing verses"):
        idx, poem_verse = future_to_task[future]

        try:
            result, failure_record = future.result()
        except Exception as e:
            result = None
            failure_record = {
                "poem_verse": poem_verse,
                "last_error": f"FutureExecutionError: {type(e).__name__}: {str(e)}",
                "last_raw_response": None,
                "timestamp": time.time(),
            }

        if result:
            with open(CONFIG["output_file"], "a") as f:
                f.write(json.dumps(result, ensure_ascii=False) + "\n")
            successful += 1
        else:
            failed += 1
            failed_payload = {
                "index": idx,
                "verse": poem_verse,
                "failure": failure_record,
            }
            failed_verses.append(failed_payload)

            with open(CONFIG["failed_output_file"], "a") as f:
                f.write(json.dumps(failed_payload, ensure_ascii=False) + "\n")

print(f"\n{'='*50}")
print("✅ Processing Complete!")
print(f"{'='*50}")
print(f"Successful: {successful}")
print(f"Failed: {failed}")
print(f"Output saved to: {CONFIG['output_file']}")
print(f"Failures saved to: {CONFIG['failed_output_file']}")

if failed_verses:
    print(f"\n⚠️  {len(failed_verses)} verses failed to process:")
    for item in failed_verses[:5]:
        print(f"  - Index {item['index']}: {item['verse'][:50]}...")

🚀 Running 199002 requests with concurrency=20


Processing verses:   3%|▎         | 6557/199002 [44:43<20:50:50,  2.56it/s]

⚠️  No JSON object found in response
Full response:
{
  "meaning": "From the moment the Senate gained authority, both men and women became aware of its influence and the political power it wielded.",

  "queries": {
    "neutral": [
      "How did the Senate’s rise to power


Processing verses:   5%|▍         | 9873/199002 [1:07:36<14:36:07,  3.60it/s]

⚠️  No JSON object found in response
Full response:
{
  "meaning": "I assert my right to be included in the count or recognition of something important, whether it’s a group, a legacy, or a meaningful contribution.",

  "queries": {
    "neutral": [
      "What does it mean to 'claim a part' in a collective effort or legacy?",
      "How can someone assert their place in a shared


Processing verses:   5%|▌         | 10893/199002 [1:14:39<29:11:38,  1.79it/s]

⚠️  No JSON object found in response
Full response:
{
  "meaning": "The baby’s innocent, joyful smile disarmed even those who disliked or opposed her, softening their hostility with its purity and charm.",

  "queries": {
    "neutral": [
      "How can a baby’s smile change someone’s attitude toward them?",
      "What does it mean when a child’s happiness seems to melt away anger or resentment?",
      "Describe a moment where innocence had


Processing verses:   7%|▋         | 13468/199002 [1:32:53<21:19:33,  2.42it/s]


## Cell 7: Validate Output

Load and inspect the generated dataset.

In [ ]:
# Load generated dataset
generated_data = []
with open(CONFIG["output_file"], 'r') as f:
    for line in f:
        generated_data.append(json.loads(line))

print(f"📊 Total entries in output file: {len(generated_data)}")

if generated_data:
    print(f"\n{'='*50}")
    print("Sample Entry:")
    print(f"{'='*50}")
    sample = generated_data[0]
    print(f"Poem Verse: {sample['poem_verse']}\n")
    print(f"Meaning: {sample['data']['meaning']}\n")
    print(f"Neutral Queries ({len(sample['data']['queries']['neutral'])}):")
    for i, q in enumerate(sample['data']['queries']['neutral'], 1):
        print(f"  {i}. {q}")
    print(f"\nUser Queries ({len(sample['data']['queries']['user'])}):")
    for i, q in enumerate(sample['data']['queries']['user'], 1):
        print(f"  {i}. {q}")

## Cell 8: Export Statistics

Generate basic statistics about the generated dataset.

In [ ]:
if generated_data:
    # Calculate statistics
    total_queries = len(generated_data) * 10  # 5 neutral + 5 user queries per entry
    
    # Average query lengths
    all_neutral_queries = [q for entry in generated_data for q in entry['data']['queries']['neutral']]
    all_user_queries = [q for entry in generated_data for q in entry['data']['queries']['user']]
    
    avg_neutral_length = sum(len(q.split()) for q in all_neutral_queries) / len(all_neutral_queries)
    avg_user_length = sum(len(q.split()) for q in all_user_queries) / len(all_user_queries)
    
    print(f"{'='*50}")
    print("Dataset Statistics")
    print(f"{'='*50}")
    print(f"Total Entries: {len(generated_data)}")
    print(f"Total Queries Generated: {total_queries}")
    print(f"Average Neutral Query Length: {avg_neutral_length:.1f} words")
    print(f"Average User Query Length: {avg_user_length:.1f} words")
    print(f"\nQuery Length Distribution (Neutral):")
    lengths = [len(q.split()) for q in all_neutral_queries]
    print(f"  Min: {min(lengths)} words")
    print(f"  Max: {max(lengths)} words")
    print(f"  Median: {sorted(lengths)[len(lengths)//2]} words")
else:
    print("⚠️  No data generated")

## Next Steps

1. **Review Quality:** Manually inspect a sample of the generated queries to ensure quality.
2. **Convert to Training Format:** Transform this data into the format required by your fine-tuning pipeline (e.g., conversational format for chat models).
3. **Split Dataset:** Create train/validation/test splits.
4. **Train Model:** Use the generated dataset in `02_Trainer_Arena.ipynb`.